# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2026-01-08 02:14:06] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2026-01-08 02:14:06] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2026-01-08 02:14:06] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2026-01-08 02:14:09] INFO server_args.py:1615: Attention backend not specified. Use fa3 backend by default.


[2026-01-08 02:14:09] INFO server_args.py:2502: Set soft_watchdog_timeout since in CI


...

Capturing batches (bs=128 avail_mem=55.35 GB):   0%|          | 0/20 [00:00<?, ?it/s]

Capturing batches (bs=80 avail_mem=55.23 GB):  20%|██        | 4/20 [00:00<00:01, 13.67it/s]

Capturing batches (bs=40 avail_mem=55.20 GB):  50%|█████     | 10/20 [00:00<00:00, 20.34it/s]

Capturing batches (bs=12 avail_mem=55.18 GB):  65%|██████▌   | 13/20 [00:00<00:00, 21.68it/s]

Capturing batches (bs=1 avail_mem=55.16 GB): 100%|██████████| 20/20 [00:01<00:00, 19.83it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Alex and I'm a 15-year-old Scottish girl who is really good at talking about some of the things that make me happy and sad. I used to be really good at drawing and I love to write poetry. I have a really neat little interest in vintage music and I often have to use my imagination to come up with new ideas. 

I also enjoy playing frisbee and having parties with my friends and I love to write some of the songs on the concert music I make. 

Now, I am going to ask you questions, but I'm not sure if I'm ready to answer them yet. I'm sorry
Prompt: The president of the United States is
Generated text:  seeking an answer to the question: "What is the capital city of a country with a population of 2,500,000?" What is the answer?

The capital city of a country with a population of 2,500,000 is Washington, D.C. 

To break it down:

1. The question is asking for the capital city of a country with a population of 2,500,000.
2. The capital city of the Unit

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name], and I'm a [Job Title] at [Company Name]. I'm excited to meet you and learn more about your interests and passions. What can you tell me about yourself? I'm a [insert a short, positive description of your personality or skills]. And what's your favorite hobby or activity? I love [insert a short, positive description of your hobby or activity]. And what's your favorite book or movie? I love [insert a short, positive description of your favorite book or movie]. And what's your favorite place to go? I love [insert a short, positive description of your favorite place]. And

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris, the city known for its iconic Eiffel Tower and its rich history dating back to the Middle Ages. It is the largest city in France and the third-largest city in the world by population. Paris is also the capital of the French department of Paris and the seat of the French government. The city is famous for its art, music, and cuisine, and is home to many world-renowned landmarks such as the Louvre Museum and the Notre-Dame Cathedral. Paris is a cultural and economic hub of France and a major tourist destination. It is also known for its fashion industry, with many famous designers and fashion houses operating in

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by rapid advancements in areas such as machine learning, natural language processing, and computer vision. These technologies will continue to improve, leading to more sophisticated and accurate AI systems that can perform a wide range of tasks with increasing accuracy and efficiency. Some potential future trends in AI include:

1. Increased integration with human intelligence: As AI becomes more sophisticated, it is likely to become more integrated with human intelligence, allowing for more complex and nuanced interactions between humans and machines.

2. Greater emphasis on ethical considerations: As AI systems become more sophisticated, there will be a greater emphasis on ethical considerations, including issues such as bias, privacy



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  Jane. I'm a freelance writer and editor, with a degree in English and a passion for storytelling. I love exploring the world through writing and have written numerous books and articles. I'm a listener and listener to listeners, and I love helping others find their voice and express themselves in their own unique way. I'm always looking for new opportunities to learn and grow as a writer and editor. Thanks for listening. [Your Name] [Your Address] [Your Phone Number] [Your Email Address] [Your LinkedIn Profile Link] [Your Portfolio Link] [Your Twitter Profile Link] [Your Instagram Profile Link] [Your Facebook Profile

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris. 

(A) The capital of France is Paris. 
(B) Paris is the capital of France. 
(C) The capital of France is Paris. 
(D) Paris is the capital.



### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

insert

 name

].

...

 diagnostic

 tools,

 but

 it

 has

In [6]:
llm.shutdown()